<a href="https://colab.research.google.com/github/bhavanapagadala/AI-Agent/blob/main/Medical_Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Medical Image

In [1]:
!pip install agno
!pip install duckduckgo-search
!pip install pillow
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.8/962.8 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 46.8 MB/s eta 0:00:00


In [2]:
import os
from PIL import Image as PILImage
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.media import Image as AgnoImage
import gradio as gr

# Load Google API Key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Prompt for medical image analysis
query = """
You are a highly skilled medical imaging expert with extensive knowledge in radiology and diagnostic imaging. Analyze the patient's medical image and structure your response as follows:

### 1. Image Type & Region
- Specify imaging modality (X-ray/MRI/CT/Ultrasound/etc.)
- Identify the patient's anatomical region and positioning
- Comment on image quality and technical adequacy

### 2. Key Findings
- List primary observations systematically
- Note any abnormalities in the patient's imaging with precise descriptions
- Include measurements and densities where relevant
- Describe location, size, shape, and characteristics
- Rate severity: Normal/Mild/Moderate/Severe

### 3. Diagnostic Assessment
- Provide primary diagnosis with confidence level
- List differential diagnoses in order of likelihood
- Support each diagnosis with observed evidence from the patient's imaging
- Note any critical or urgent findings

### 4. Patient-Friendly Explanation
- Explain the findings in simple, clear language that the patient can understand
- Avoid medical jargon or provide clear definitions
- Include visual analogies if helpful
- Address common patient concerns related to these findings

### 5. Research Context
IMPORTANT: Use the DuckDuckGo search tool to:
- Find recent medical literature about similar cases
- Search for standard treatment protocols
- Provide a list of relevant medical links of them too
- Research any relevant technological advances
- Include 2-3 key references to support your analysis

Format your response using clear markdown headers and bullet points. Be concise yet thorough.
"""

# Image analysis function
def analyze_medical_image(api_key, image_np):
    if not api_key:
        return "❌ Please provide a valid Google API key."

    # Initialize the agent
    agent = Agent(
        model=Gemini(id="gemini-2.0-flash", api_key=api_key),
        tools=[DuckDuckGoTools()],
        markdown=True
    )

    # Resize image
    img = PILImage.fromarray(image_np)
    img = img.resize((500, int(500 / img.width * img.height)))
    temp_path = "temp_image.png"
    img.save(temp_path)


    # Analyze with Gemini + DuckDuckGo
    agno_img = AgnoImage(filepath=temp_path)
    result = agent.run(query, images=[agno_img])

    return result.content


# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🏥 Medical Imaging Diagnosis Agent")
    gr.Markdown("Upload a medical image to receive an AI-generated analysis report.\n\n⚠️ **Disclaimer:** Educational use only. Not a clinical diagnosis tool.")

    api_input = gr.Textbox(label="🔑 Google API Key", type="password", placeholder="Paste your API key")
    image_input = gr.Image(label="🖼️ Upload Medical Image", type="numpy")
    analyze_button = gr.Button("🔍 Analyze")
    output_md = gr.Markdown()

    analyze_button.click(analyze_medical_image, inputs=[api_input, image_input], outputs=output_md)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://04dc9ac7771347ee79.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install agno
!pip install duckduckgo-search

In [ ]:
import os
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools.duckduckgo import DuckDuckGoTools
import gradio as gr

# Load Google API Key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Assistant prompt
assistant_prompt = """
You are an intelligent, friendly, and highly skilled personal assistant who helps users manage their daily tasks, knowledge, productivity, and general inquiries. Respond in a professional, friendly, and concise tone. Use markdown formatting.

### 🧾 1. Task Understanding
- Clarify the user's goal or request.
- Rephrase it briefly in your own words.
- Ask for clarifications if needed.

### ✅ 2. Solution / Response
- Provide the most accurate and helpful response.
- Use bullet points or steps where appropriate.
- Offer suggestions, alternatives, or actions.
- If the request includes a topic or query, perform a DuckDuckGo search to:
  - Retrieve the most relevant, recent, and authoritative information.
  - Include 2–3 links with brief descriptions.

### 🧘 3. Friendly Summary
- End with a short, friendly message or suggestion for next steps.
- Keep the tone light, warm, and encouraging.
"""

# Assistant function (without try-except)
def personal_assistant(api_key, user_input):
    if not api_key:
        return "❌ Please provide a valid Google API key."

    agent = Agent(
        model=Gemini(id="gemini-2.0-flash", api_key=api_key),
        tools=[DuckDuckGoTools()],
        markdown=True
    )

    full_prompt = f"{assistant_prompt}\n\nUser Request: {user_input}"
    result = agent.run(full_prompt)
    return result.content

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🤖 Your AI Personal Assistant")
    gr.Markdown("Ask me anything — from tasks, productivity tips, summaries, general questions, or tech help.\n\n📝 **Example prompts:**\n- Plan my day with 3 tasks\n- What are the latest AI tools?\n- Summarize https://... \n- Draft an email to my boss")

    api_input = gr.Textbox(label="🔑 Google API Key", type="password", placeholder="Paste your API key")
    text_input = gr.Textbox(label="💬 Ask me anything", placeholder="e.g. Summarize today's top tech news...")
    submit_button = gr.Button("🚀 Ask Assistant")
    output_md = gr.Markdown()

    submit_button.click(personal_assistant, inputs=[api_input, text_input], outputs=output_md)

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bef10897834849c381.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools.duckduckgo import DuckDuckGoTools

In [ ]:
def personal_assistant(api_key, user_input):
    if not api_key:
      return "❌ Please provide a valid Google API key."
    agent=Agent(
        model=Gemini(id="gemini-2.0-flash", api_key=api_key),
        tools=[DuckDuckGoTools()],
        markdown=True
    )

    Img=PILImage.fromarray(image_np)
    Img=Img.resize((500, int(500/Img.width*Img.height)))
    temp_path="temp_image.png"
    Img.save(temp_path)

    agno_img=AgnoImage(filepath=temp_path)
    result=agent.run(user_input, images=[agno_img])
    return result.content

with gr.Blocks() as demo:
  gr.Markdown()




SyntaxError: expected ':' (ipython-input-6-1298197201.py, line 4)

In [ ]:
pip install duckduckgo-search

## Personal Agent

## Weather App

In [ ]:
import os
import requests
import gradio as gr
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools.duckduckgo import DuckDuckGoTools

# 🌤️ Weather summary template
PROMPT_TEMPLATE = """
You are a weather analyst. Based on real-time weather in **{location}**, write a detailed weather report:

### 1. Current Weather
- Temperature: {temperature}°C
- Wind Speed: {windspeed} km/h
- Humidity: {humidity}%
- Sky: {description}

### 2. Recommendations
- What should a person wear?
- Precautions to take based on weather?

### 3. Patient-Friendly Insight
Explain the weather conditions in simple terms and how it affects daily life.

### 4. Add 2–3 helpful weather websites using DuckDuckGo.
"""

# Open-Meteo weather code meanings
WEATHER_CODES = {
    0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
    45: "Fog", 48: "Rime fog", 51: "Light drizzle", 53: "Moderate drizzle",
    61: "Light rain", 63: "Moderate rain", 65: "Heavy rain",
    71: "Light snow", 73: "Moderate snow", 75: "Heavy snow",
    80: "Rain showers", 81: "Heavy rain showers",
    95: "Thunderstorm", 99: "Severe thunderstorm"
}

# 🌍 Get coordinates for a city
def get_coords(city):
    url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
    res = requests.get(url).json()
    if res.get("results"):
        return res["results"][0]["latitude"], res["results"][0]["longitude"]
    return None, None

# 🌦️ Get weather data from Open-Meteo
def get_weather(lat, lon):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m,windspeed_10m,weathercode&timezone=auto"
    return requests.get(url).json().get("current", {})

# 🔍 Weather Analysis Function
def analyze_weather(api_key, city):
    if not api_key:
        return "❌ Google API Key is missing."
    if not city.strip():
        return "❌ Please enter a valid city name."

    try:
        lat, lon = get_coords(city)
        if not lat:
            return "❌ City not found."

        weather = get_weather(lat, lon)
        desc = WEATHER_CODES.get(weather.get("weathercode", 0), "Unknown")

        # Format prompt
        prompt = PROMPT_TEMPLATE.format(
            location=city.title(),
            temperature=weather.get("temperature_2m", "N/A"),
            windspeed=weather.get("windspeed_10m", "N/A"),
            humidity=weather.get("relative_humidity_2m", "N/A"),
            description=desc
        )

        agent = Agent(
            model=Gemini(id="gemini-1.5-flash", api_key=api_key),
            tools=[DuckDuckGoTools()],
            markdown=True
        )

        return agent.run(prompt).content

    except Exception as e:
        return f"❌ Error: {e}"

# 🚀 Gradio App UI
with gr.Blocks() as demo:
    gr.Markdown("## 🌦️ AI Weather Reporter")
    gr.Markdown("Get live weather updates + AI analysis (powered by Open-Meteo + Gemini)\n⚠️ For demo use only.")

    api_key = gr.Textbox(label="🔐 Google API Key", type="password")
    city_name = gr.Textbox(label="📍 City Name", placeholder="e.g., Bengaluru, London")
    analyze_btn = gr.Button("🔍 Analyze Weather")
    output = gr.Markdown()

    analyze_btn.click(analyze_weather, inputs=[api_key, city_name], outputs=output)

demo.launch()
